# To Identify Terrorist Events using Event Triggers

## Imports

### Import Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Import Pre-defined Libraries/Packages/Modules/Methods

In [0]:
import pandas as pd
import numpy as np
import spacy as sp
import pickle
import os
from collections import OrderedDict
from collections import defaultdict
import string
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

### Installations/Loading

In [3]:
!pip3 install contractions
import contractions

     |████████████████████████████████| 245kB 12.6MB/s 
     |████████████████████████████████| 317kB 62.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=0eab9eb00d443ee7cd880496a5ebd0cabe22c0ce2dbffb4bcda932677bc0e932
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [4]:
!python -m spacy download en_core_web_lg
!pip3 install spacy-wordnet

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=bcbd9667caf74f657eeb4c6d8c89d7a27933ef8725c750d348831ce581ee8b20
  Stored in directory: /tmp/pip-ephem-wheel-cache-121ui8bl/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 655kB 8.1MB/s 
     |████████████████████████████████| 1.4MB 25.6MB/s 
  Created wheel for spacy-wordnet: filename=spacy_wordnet-0.0.4-py2.py3-none-any.whl size=650292 sha256=70fc004e7568711a0dd12866065c0f02243cf2db91aa6d7bb2445d7662209722
  Stored in directory: /root/.cache/pip/wheels/25/93/1d/c86db913cd146fc9ddb26d10f56579c5d58a3e00bc8f96a3a6
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394468 sha256=e9b9284d2511b45b63dc4468d92487f9195f3a579daa

In [5]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
!python -m nltk.downloader wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Import Custom Modules

## Global Objects


In [0]:
# nlp = sp.load('en_core_web_lg')

In [0]:
HOME_PATH = './drive/My Drive/Colab Notebooks/201916007_PROJECT_IT550'
DATABASES_PATH = HOME_PATH + '/DATABASES'
GTD = DATABASES_PATH + '/globalterrorismdb_0718dist.csv'
RDWTI = DATABASES_PATH + '/RAND_Database_of_Worldwide_Terrorism_Incidents.csv'
INTERMEDIATE_FILES_PATH = HOME_PATH + '/INTERMEDIATE_FILES'
FIRE_DATABASE_FILES_PATH = HOME_PATH + '/FIRE_DATABASE_FILES'
FIRE_DATA_DICT = FIRE_DATABASE_FILES_PATH + '/file_data_dict.txt'
TRIGGERS_PATH = HOME_PATH + '/TRIGGERS'
TERRORIST_EVENTS_UPDATED1_PATH = TRIGGERS_PATH+'/TERRORIST_EVENT_TRIGGERS_UPDATED1.txt'
ENCODING = 'ISO-8859-1'
ENGINE = 'python'
LEM = WordNetLemmatizer()
Encoder = LabelEncoder()
# LANCASTER = LancasterStemmer()
# PORTER = PorterStemmer()

## Exploratory Data Analysis

### GTD (Global Terrorism Database)

#### Load database

In [0]:
gtd = pd.read_csv(GTD, engine=ENGINE, encoding=ENCODING)

#### EDA

In [0]:
# print(gtd.info())

In [0]:
# print(gtd.head())

In [0]:
# print(gtd.columns.values)

In [0]:
# i,j = 0,2
# while j <= 135:
#   print(gtd.iloc[-5:, i:j])
#   i += 2
#   j += 2

In [0]:
# i,j = 0,2
# while j <= 135:
#   print(gtd.iloc[:5, i:j])
#   i += 2
#   j += 2

In [0]:
# print(gtd['suicide'].value_counts())

#### Feature Selection

In [0]:
# GTD_FEATURE_NAMES = ['alternative_txt','attacktype1_txt','attacktype2_txt',
#                      'attacktype3_txt','gname','gname2','gname3','gsubname',
#                      'gsubname2','gsubname3','scite1','scite2','scite3',
#                      'summary','target1','targtype1_txt','targsubtype1_txt',
#                      'target2','targtype2_txt','targsubtype2_txt','target3',
#                      'targtype3_txt','targsubtype3_txt','weapdetail',
#                      'weaptype1_txt','weapsubtype1_txt','weaptype2_txt',
#                      'weapsubtype2_txt','weaptype3_txt','weapsubtype3_txt',
#                      'weaptype4_txt','weapsubtype4_txt']

In [0]:
# gtd_view1 = gtd[GTD_FEATURE_NAMES]

In [0]:
# EXTRACT COLUMNS WHICH HAVE ATTACK TYPES
# ATTACK_TYPE_COLS = ['attacktype1_txt','attacktype2_txt','attacktype3_txt']
# gtd_view_attacks = gtd_view1.dropna(
#     how='all',
#     subset=ATTACK_TYPE_COLS)

In [0]:
# EXTRACT UNIQUE ATTACK TYPES
# ATTACK_TYPES = [attacktype for attacktype in
#                 set().union(*[set(gtd_view_attacks[attacktype_col])
#                 for attacktype_col in ATTACK_TYPE_COLS])
#                 if attacktype==attacktype]  # We know that, nan != nan

In [0]:
# ATTACK_TYPES

In [0]:
# EXTRACT COLUMNS WHICH HAVE WEAPON TYPES ## 'weapdetail', 
# WEAPON_TYPE_COLS = ['weaptype1_txt','weapsubtype1_txt',
#                     'weaptype2_txt','weapsubtype2_txt','weaptype3_txt',
#                     'weapsubtype3_txt','weaptype4_txt','weapsubtype4_txt']

# gtd_view_weapons = gtd_view1.dropna(
#     how='all',
#     subset=WEAPON_TYPE_COLS)

In [0]:
# EXTRACT UNIQUE WEAPON TYPES
# WEAPON_TYPES = [weapontype for weapontype in
#                 set().union(*[set(gtd_view_weapons[weapontype_col])
#                 for weapontype_col in WEAPON_TYPE_COLS])
#                 if weapontype==weapontype] # We know that, nan != nan

In [0]:
# WEAPON_TYPES

## Prepare data for classification

### GTD

#### Extract Columns

In [0]:
gtd_view = gtd[['attacktype1_txt', 'summary']].dropna()

#### Preprocessing

In [0]:
def preprocessing(data):
    """Function to preprocess the raw data"""
    
    global LEM

    # Remove contractions
    contractions_removed = contractions.fix(data.lower(), leftovers=True, slang=True)
    
    # Remove urls
    url_regex = r'\b((?:https?://)+(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b'
    urls_extracted = re.findall(url_regex, contractions_removed)
    contractions_removed = re.sub(url_regex, " ", contractions_removed, flags=re.MULTILINE)
    
    # Remove punctuations and digits
    punc_num_to_be_removed = string.punctuation+string.digits
    punctuation_removed = ''.join([char for char in contractions_removed if char not in punc_num_to_be_removed])
    
    # Remove multiple whitespace
    multi_whitespace_removed = re.sub(r'\s+', ' ', punctuation_removed)

    # Tokenization
    tokens = word_tokenize(multi_whitespace_removed)

    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    Final_words = []
    for word, tag in pos_tag(tokens):
      if word not in stopwords.words('english') and word.isalpha():
        word_Final = LEM.lemmatize(word,tag_map[tag[0]])
        Final_words.append(word_Final)


    return Final_words
    

In [0]:
# for idx, entry in enumerate(gtd_view['summary']):
#   gtd_view.loc[idx, 'summary_new'] = str(preprocessing(entry))

In [0]:
# pickle.dump(gtd_view['summary_new'], open(INTERMEDIATE_FILES_PATH+'/gtd_view.txt', 'wb'))
gtd_view['summary_new'] = pickle.load(open(INTERMEDIATE_FILES_PATH+'/gtd_view.txt', 'rb'))

### Split Data

In [0]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.\
train_test_split(gtd_view['summary_new'],
                 gtd_view['attacktype1_txt'],
                 test_size=0.3)

In [0]:
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))

### TFIDF

In [0]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(gtd_view['summary_new'].astype(str))
Train_X_Tfidf = Tfidf_vect.transform(Train_X.astype(str))
Test_X_Tfidf = Tfidf_vect.transform(Test_X.astype(str))

## Classification

### Naive Bayes Classifier

In [0]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)

In [0]:
print(classification_report(predictions_NB, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.38      0.00        52
           1       0.00      0.00      0.00         0
           2       1.00      0.53      0.69     34613
           3       0.00      0.50      0.00         4
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.53     34669
   macro avg       0.11      0.16      0.08     34669
weighted avg       1.00      0.53      0.69     34669



In [57]:
print(precision_score(predictions_NB, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_NB, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_NB, Test_Y, zero_division=0, average='weighted'))

0.9972933914033522
0.5259742132741065
0.6881800306620117


### SVM Classifier

In [0]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [60]:
print(classification_report(predictions_SVM, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.33      0.01        73
           1       0.00      0.67      0.00         3
           2       1.00      0.53      0.69     34587
           3       0.00      0.17      0.00         6
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.53     34669
   macro avg       0.11      0.19      0.08     34669
weighted avg       1.00      0.53      0.69     34669



In [61]:
print(precision_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))

0.9955619514045915
0.5255992385127926
0.6873267249805707


In [0]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [65]:
print(classification_report(predictions_SVM, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.53      0.69     34669
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.53     34669
   macro avg       0.11      0.06      0.08     34669
weighted avg       1.00      0.53      0.69     34669



In [66]:
print(precision_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))

1.0
0.525916524849289
0.6893123133340894


### Random Forest

In [0]:
RF = RandomForestClassifier()
RF.fit(Train_X_Tfidf,Train_Y)
predictions_RF = RF.predict(Test_X_Tfidf)

In [68]:
print(classification_report(predictions_RF, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.02      0.29      0.04       662
           1       0.00      0.11      0.01        88
           2       0.98      0.53      0.68     33751
           3       0.01      0.27      0.02        77
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         2
           6       0.00      0.21      0.01        58
           7       0.00      0.00      0.00         5
           8       0.00      0.10      0.00        21

    accuracy                           0.52     34669
   macro avg       0.11      0.17      0.09     34669
weighted avg       0.95      0.52      0.67     34669



In [69]:
print(precision_score(predictions_RF, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_RF, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_RF, Test_Y, zero_division=0, average='weighted'))

0.9501501172265656
0.5198592402434451
0.6670890797870292


### XGBoost Classifier

In [0]:
XGB = XGBClassifier()
XGB.fit(Train_X_Tfidf,Train_Y)
predictions_XGB = XGB.predict(Test_X_Tfidf)

In [26]:
print(classification_report(predictions_XGB, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.50      0.00         8
           1       0.00      1.00      0.00         3
           2       1.00      0.52      0.69     34641
           3       0.00      0.24      0.00        17
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.52     34669
   macro avg       0.11      0.25      0.08     34669
weighted avg       1.00      0.52      0.69     34669



In [27]:
print(precision_score(predictions_XGB, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_XGB, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_XGB, Test_Y, zero_division=0, average='weighted'))

0.9985325953222426
0.523147480458046
0.6863119231052144


## Classification N-Gram (2,3)

In [0]:
Tfidf_vect = TfidfVectorizer(ngram_range=(2,3))
Tfidf_vect.fit(gtd_view['summary_new'].astype(str))
Train_X_Tfidf = Tfidf_vect.transform(Train_X.astype(str))
Test_X_Tfidf = Tfidf_vect.transform(Test_X.astype(str))

### Naive Bayes Classifier

In [0]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)

In [72]:
print(classification_report(predictions_NB, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.27      0.00        15
           1       0.00      1.00      0.00         2
           2       1.00      0.53      0.69     34650
           3       0.00      1.00      0.00         2
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.53     34669
   macro avg       0.11      0.31      0.08     34669
weighted avg       1.00      0.53      0.69     34669



In [73]:
print(precision_score(predictions_NB, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_NB, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_NB, Test_Y, zero_division=0, average='weighted'))

0.9991233931675582
0.5259742132741065
0.6889559259017372


### SVM Classifier

In [0]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [75]:
print(classification_report(predictions_SVM, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.01      0.33      0.02       265
           1       0.00      0.60      0.00         5
           2       0.99      0.53      0.69     34377
           3       0.00      0.44      0.01        16
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.60      0.00         5
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1

    accuracy                           0.52     34669
   macro avg       0.11      0.28      0.08     34669
weighted avg       0.98      0.52      0.68     34669



In [76]:
print(precision_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))

0.9844291965141213
0.5249935100522081
0.6824509616062843


In [0]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [78]:
print(classification_report(predictions_SVM, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.53      0.69     34669
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.53     34669
   macro avg       0.11      0.06      0.08     34669
weighted avg       1.00      0.53      0.69     34669



In [79]:
print(precision_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_SVM, Test_Y, zero_division=0, average='weighted'))

1.0
0.525916524849289
0.6893123133340894


### Random Forest

In [0]:
RF = RandomForestClassifier()
RF.fit(Train_X_Tfidf,Train_Y)
predictions_RF = RF.predict(Test_X_Tfidf)

In [81]:
print(classification_report(predictions_RF, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.02      0.28      0.04       674
           1       0.00      0.12      0.01        72
           2       0.97      0.53      0.68     33752
           3       0.01      0.24      0.02        79
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         1
           6       0.00      0.14      0.01        65
           7       0.00      0.00      0.00         4
           8       0.00      0.06      0.00        17

    accuracy                           0.52     34669
   macro avg       0.11      0.15      0.08     34669
weighted avg       0.95      0.52      0.67     34669



In [82]:
print(precision_score(predictions_RF, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_RF, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_RF, Test_Y, zero_division=0, average='weighted'))

0.949471273648215
0.5191092907208169
0.6665836940202605


### XGBoost Classifier

In [0]:
XGB = XGBClassifier()
XGB.fit(Train_X_Tfidf,Train_Y)
predictions_XGB = XGB.predict(Test_X_Tfidf)

In [86]:
print(classification_report(predictions_XGB, Test_Y, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.33      0.00         6
           1       0.00      1.00      0.00         1
           2       1.00      0.53      0.69     34653
           3       0.00      0.56      0.01         9
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.53     34669
   macro avg       0.11      0.27      0.08     34669
weighted avg       1.00      0.53      0.69     34669



In [87]:
print(precision_score(predictions_XGB, Test_Y, zero_division=0, average='weighted'))
print(recall_score(predictions_XGB, Test_Y, zero_division=0, average='weighted'))
print(f1_score(predictions_XGB, Test_Y, zero_division=0, average='weighted'))

0.9992651304303738
0.5260030574865153
0.6890151106249981
